In [ ]:
#!import Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

...

Installing Packages Expecto........

Installing Packages Expecto.........

Installing Packages Expecto..........

Installing Packages Expecto...........

Installed Packages Expecto, 10.1.0

## Parser

In [ ]:
open System

### TextInput

In [ ]:
type Position =
    {
        line : int
        column : int
    }

In [ ]:
let initialPos = { line = 0; column = 0 }

In [ ]:
let incrCol (pos : Position) =
    { pos with column = pos.column + 1 }

In [ ]:
let incrLine pos =
    { line = pos.line + 1; column = 0 }

In [ ]:
type InputState =
    {
        lines : string[]
        position : Position
    }

In [ ]:
let fromStr str =
    if String.IsNullOrEmpty str then
        { lines = [||]; position = initialPos }
    else
        let separators = [| "\r\n"; "\n" |]
        let lines = str.Split (separators, StringSplitOptions.None)
        { lines = lines; position = initialPos }

In [ ]:
//// test

fromStr "" |> _equal {
    lines = [||]
    position = { line = 0; column = 0 }
}

InputState
      lines: [  ]
      position: Position
        line: 0
        column: 0


In [ ]:
//// test

fromStr "Hello \n World" |> _equal {
    lines = [| "Hello "; " World" |]
    position = { line = 0; column = 0 }
}

InputState
      lines: [ Hello ,  World ]
      position: Position
        line: 0
        column: 0


In [ ]:
let currentLine inputState =
    let linePos = inputState.position.line
    if linePos < inputState.lines.Length then
        inputState.lines.[linePos]
    else
        "end of file"

In [ ]:
let nextChar input =
    let linePos = input.position.line
    let colPos = input.position.column

    if linePos >= input.lines.Length then
        input, None
    else
        let currentLine = currentLine input
        if colPos < currentLine.Length then
            let char = currentLine.[colPos]
            let newPos = incrCol input.position
            let newState = { input with position = newPos }
            newState, Some char
        else
            let char = '\n'
            let newPos = incrLine input.position
            let newState = { input with position = newPos }
            newState, Some char

In [ ]:
//// test

let newInput, charOpt = fromStr "Hello World" |> nextChar

newInput |> _equal {
    lines = [| "Hello World" |]
    position = { line = 0; column = 1 }
}
charOpt |> _equal (Some 'H')

InputState
      lines: [ Hello World ]
      position: Position
        line: 0
        column: 1
FSharpOption<Char>
      Value: H


In [ ]:
//// test

let newInput, charOpt = fromStr "Hello\n\nWorld" |> nextChar

newInput |> _equal {
    lines = [| "Hello"; ""; "World" |]
    position = { line = 0; column = 1 }
}
charOpt |> _equal (Some 'H')

InputState
      lines: [ Hello, , World ]
      position: Position
        line: 0
        column: 1
FSharpOption<Char>
      Value: H


### Parser

In [ ]:
type Input = InputState
type ParserLabel = string
type ParserError = string

type ParserPosition =
    {
        currentLine : string
        line : int
        column : int
    }

type ParseResult<'a> =
    | Success of 'a
    | Failure of ParserLabel * ParserError * ParserPosition

type Parser<'a> =
    {
        parseFn : Input -> ParseResult<'a * Input>
        label : ParserLabel
    }

In [ ]:
let printResult result =
    match result with
    | Success (value, input) ->
        printfn $"%A{value}"
    | Failure (label, error, parserPos) ->
        let errorLine = parserPos.currentLine
        let colPos = parserPos.column
        let linePos = parserPos.line
        let failureCaret = $"{' '.ToString().PadLeft colPos}^{error}"
        printfn $"Line:%i{linePos} Col:%i{colPos} Error parsing %s{label}\n%s{errorLine}\n%s{failureCaret}"

In [ ]:
//// ignore

let parserEqual (expected : ParseResult<'a>) (actual : ParseResult<'a * Input>) =
    match actual, expected with
    | Success (_actual, _), Success _expected ->
        printResult actual
        _actual |> _equal _expected
    | Failure (l1, e1, p1), Failure (l2, e2, p2) when l1 = l2 && e1 = e2 && p1 = p2 ->
        printResult actual
    | _ ->
        printfn $"Actual: {actual}"
        printfn $"Expected: {expected}"
        failwith "Parse failed"
    actual

In [ ]:
let runOnInput parser input =
    parser.parseFn input

In [ ]:
let run parser inputStr =
    runOnInput parser (fromStr inputStr)

In [ ]:
let parserPositionFromInputState (inputState : Input) =
    {
        currentLine = currentLine inputState
        line = inputState.position.line
        column = inputState.position.column
    }

In [ ]:
let getLabel parser =
    parser.label

In [ ]:
let setLabel parser newLabel =
    let newInnerFn input =
        match parser.parseFn input with
        | Success s -> Success s
        | Failure (oldLabel, err, pos) -> Failure (newLabel, err, pos)

    { parseFn = newInnerFn; label = newLabel }

In [ ]:
let (<?>) = setLabel

In [ ]:
let satisfy predicate label =
    let innerFn input =
        let remainingInput, charOpt = nextChar input
        match charOpt with
        | None ->
            let err = "No more input"
            let pos = parserPositionFromInputState input
            Failure (label, err, pos)
        | Some first ->
            if predicate first then
                Success (first, remainingInput)
            else
                let err = $"Unexpected '%c{first}'"
                let pos = parserPositionFromInputState input
                Failure (label, err, pos)

    { parseFn = innerFn; label = label }

In [ ]:
//// test

let input = fromStr "Hello"
let parser = satisfy (fun c -> c = 'H') "H"
runOnInput parser input |> _equal (
    Success (
        'H',
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 1 }
        }
    )
)

Success
      Item:       - H
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 1


In [ ]:
//// test

let input = fromStr "World"
let parser = satisfy (fun c -> c = 'H') "H"
runOnInput parser input |> _equal (
    Failure (
        "H",
        "Unexpected 'W'",
        {
            currentLine = "World"
            line = 0
            column = 0
        }
    )
)

Failure
      Item1: H
      Item2: Unexpected 'W'
      Item3: ParserPosition
        currentLine: World
        line: 0
        column: 0


In [ ]:
let bindP f p =
    let label = "unknown"
    let innerFn input =
        let result1 = runOnInput p input
        match result1 with
        | Failure (label, err, pos) ->
            Failure (label, err, pos)
        | Success (value1, remainingInput) ->
            let p2 = f value1
            runOnInput p2 remainingInput

    { parseFn = innerFn; label = label }

In [ ]:
let (>>=) p f = bindP f p

In [ ]:
//// test

let input = fromStr "Hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = parser >>= fun c -> satisfy (fun c -> c = 'e') "e"
runOnInput parser2 input |> _equal (
    Success (
        'e',
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 2 }
        }
    )
)

Success
      Item:       - e
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 2


In [ ]:
//// test

let input = fromStr "World"
let parser = satisfy (fun c -> c = 'W') "W"
let parser2 = parser >>= fun c -> satisfy (fun c -> c = 'e') "e"
runOnInput parser2 input |> _equal (
    Failure (
        "e",
        "Unexpected 'o'",
        {
            currentLine = "World"
            line = 0
            column = 1
        }
    )
)

Failure
      Item1: e
      Item2: Unexpected 'o'
      Item3: ParserPosition
        currentLine: World
        line: 0
        column: 1


In [ ]:
let returnP x =
    let label = $"%A{x}"
    let innerFn input =
        Success (x, input)
    
    { parseFn = innerFn; label = label }

In [ ]:
//// test

let input = fromStr "Hello"
let parser = returnP "Hello"
runOnInput parser input |> _equal (
    Success (
        "Hello",
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 0 }
        }
    )
)

Success
      Item:       - Hello
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 0


In [ ]:
let mapP f =
    bindP (f >> returnP)

In [ ]:
let (<!>) = mapP

In [ ]:
let (|>>) x f = f <!> x

In [ ]:
//// test

let input = fromStr "Hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = parser |>> fun c -> c.ToString()
runOnInput parser2 input |> _equal (
    Success (
        "H",
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 1 }
        }
    )
)

Success
      Item:       - H
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 1


In [ ]:
let applyP fP xP =
    fP >>=
        fun f ->
            xP >>=
                fun x ->
                    returnP (f x)

In [ ]:
let (<*>) = applyP

In [ ]:
let lift2 f xP yP =
    returnP f <*> xP <*> yP

In [ ]:
//// test

let input = fromStr "Hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = satisfy (fun c -> c = 'e') "e"
let parser3 = lift2 (fun c1 c2 -> c1.ToString () + c2.ToString ()) parser parser2
runOnInput parser3 input |> _equal (
    Success (
        "He",
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 2 }
        }
    )
)

Success
      Item:       - He
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 2


In [ ]:
let andThen p1 p2 =
    p1 >>=
        fun p1Result ->
            p2 >>=
                fun p2Result ->
                    returnP (p1Result, p2Result)
    <?> $"{getLabel p1} andThen {getLabel p2}"

In [ ]:
let (.>>.) = andThen

In [ ]:
//// test

let input = fromStr "Hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = satisfy (fun c -> c = 'e') "e"
let parser3 = parser .>>. parser2
runOnInput parser3 input |> _equal (
    Success (
        ('H', 'e'),
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 2 }
        }
    )
)

Success
      Item:       - ( H, e )
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 2


In [ ]:
let orElse p1 p2 =
    let label = $"{getLabel p1} orElse {getLabel p2}"
    let innerFn input =
        let result1 = runOnInput p1 input

        match result1 with
        | Success result -> result1
        | Failure _ -> runOnInput p2 input

    { parseFn = innerFn; label = label }

In [ ]:
let (<|>) = orElse

In [ ]:
//// test

let input = fromStr "hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = satisfy (fun c -> c = 'h') "h"
let parser3 = parser <|> parser2
runOnInput parser3 input |> _equal (
    Success (
        'h',
        {
            lines = [| "hello" |]
            position = { line = 0; column = 1 }
        }
    )
)

Success
      Item:       - h
      - InputState
          lines: [ hello ]
          position: Position
            line: 0
            column: 1


In [ ]:
let choice listOfParsers =
    listOfParsers |> List.reduce (<|>)

In [ ]:
//// test

let input = fromStr "hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = satisfy (fun c -> c = 'h') "h"
let parser3 = choice [parser; parser2]
runOnInput parser3 input |> _equal (
    Success (
        'h',
        {
            lines = [| "hello" |]
            position = { line = 0; column = 1 }
        }
    )
)

Success
      Item:       - h
      - InputState
          lines: [ hello ]
          position: Position
            line: 0
            column: 1


In [ ]:
let rec sequence parserList =
    let cons head tail = head :: tail

    let consP = lift2 cons

    match parserList with
    | [] -> returnP []
    | head :: tail -> consP head (sequence tail)

In [ ]:
//// test

let input = fromStr "Hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = satisfy (fun c -> c = 'e') "e"
let parser3 = sequence [parser; parser2]
runOnInput parser3 input |> _equal (
    Success (
        ['H'; 'e'],
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 2 }
        }
    )
)

Success
      Item:       - [ H, e ]
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 2


In [ ]:
let rec parseZeroOrMore parser input =
    let firstResult = runOnInput parser input
    match firstResult with
    | Failure (_, _, _) ->
        [], input
    | Success (firstValue, inputAfterFirstParse) ->
        let (subsequentValues, remainingInput) =
            parseZeroOrMore parser inputAfterFirstParse
        let values = firstValue :: subsequentValues
        values, remainingInput

In [ ]:
let many parser =
    let label = $"many {getLabel parser}"
    let innerFn input =
        Success (parseZeroOrMore parser input)
    { parseFn = innerFn; label = label }

In [ ]:
//// test

let input = fromStr "hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = many parser
runOnInput parser2 input |> _equal (
    Success (
        [],
        {
            lines = [| "hello" |]
            position = { line = 0; column = 0 }
        }
    )
)

Success
      Item:       - [  ]
      - InputState
          lines: [ hello ]
          position: Position
            line: 0
            column: 0


In [ ]:
let many1 p =
    p >>=
        fun head ->
            many p >>=
                fun tail ->
                    returnP (head :: tail)
    <?> $"many1 {getLabel p}"

In [ ]:
//// test

let input = fromStr "hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = many1 parser
runOnInput parser2 input |> _equal (
    Failure (
        "many1 H",
        "Unexpected 'h'",
        {
            currentLine = "hello"
            line = 0
            column = 0
        }
    )
)

Failure
      Item1: many1 H
      Item2: Unexpected 'h'
      Item3: ParserPosition
        currentLine: hello
        line: 0
        column: 0


In [ ]:
let opt p =
    let some = p |>> Some
    let none = returnP None
    (some <|> none)
    <?> $"opt {getLabel p}"

In [ ]:
//// test

let input = fromStr "hello"
let parser = satisfy (fun c -> c = 'H') "H"
let parser2 = opt parser
runOnInput parser2 input |> _equal (
    Success (
        None,
        {
            lines = [| "hello" |]
            position = { line = 0; column = 0 }
        }
    )
)

Success
      Item:       - <null>
      - InputState
          lines: [ hello ]
          position: Position
            line: 0
            column: 0


In [ ]:
let (.>>) p1 p2 =
    p1 .>>. p2
    |> mapP fst

In [ ]:
let (>>.) p1 p2 =
    p1 .>>. p2
    |> mapP snd

In [ ]:
let between p1 p2 p3 =
    p1 >>. p2 .>> p3

In [ ]:
//// test

let input = fromStr "[Hello]"
let parser =
    between
        (satisfy (fun c -> c = '[') "[")
        (many (satisfy (fun c -> ['a'..'z'] @ ['A'..'Z'] |> List.contains c) "letter"))
        (satisfy (fun c -> c = ']') "]")
runOnInput parser input |> _equal (
    Success (
        ['H'; 'e'; 'l'; 'l'; 'o'],
        {
            lines = [| "[Hello]" |]
            position = { line = 0; column = 7 }
        }
    )
)

Success
      Item:       - [ H, e, l, l, o ]
      - InputState
          lines: [ [Hello] ]
          position: Position
            line: 0
            column: 7


In [ ]:
let sepBy1 p sep =
    let sepThenP = sep >>. p
    p .>>. many sepThenP
    |>> fun (p, pList) -> p :: pList

In [ ]:
let sepBy p sep =
    sepBy1 p sep <|> returnP []

In [ ]:
//// test

let input = fromStr "Hello,World"
let parser = sepBy (many (satisfy (fun c -> c <> ',') "not comma")) (satisfy (fun c -> c = ',') "comma")
runOnInput parser input |> _equal (
    Success (
        [['H'; 'e'; 'l'; 'l'; 'o']; ['W'; 'o'; 'r'; 'l'; 'd'; '\n']],
        {
            lines = [| "Hello,World" |]
            position = { line = 1; column = 0 }
        }
    )
)

Success
      Item:       - FSharpList<FSharpList<Char>>
[ H, e, l, l, o ]
[ W, o, r, l, d, 
 ]
      - InputState
          lines: [ Hello,World ]
          position: Position
            line: 1
            column: 0


In [ ]:
let pchar charToMatch =
    let predicate ch = ch = charToMatch
    satisfy predicate $"%c{charToMatch}"

In [ ]:
let anyOf listOfChars =
    listOfChars
    |> List.map pchar
    |> choice
    <?> $"anyOf %A{listOfChars}"

In [ ]:
//// test

let input = fromStr "Hello"
let parser = anyOf ['H'; 'e'; 'l'; 'o'] |> many
runOnInput parser input |> _equal (
    Success (
        ['H'; 'e'; 'l'; 'l'; 'o'],
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 5 }
        }
    )
)

Success
      Item:       - [ H, e, l, l, o ]
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 5


In [ ]:
let charListToStr charList =
    charList |> List.toArray |> String

In [ ]:
let manyChars cp =
    many cp
    |>> charListToStr

In [ ]:
let manyChars1 cp =
    many1 cp
    |>> charListToStr

In [ ]:
//// test

let input = fromStr "Hello"
let parser = manyChars1 (anyOf ['H'; 'e'; 'l'; 'o'])
runOnInput parser input |> _equal (
    Success (
        "Hello",
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 5 }
        }
    )
)

Success
      Item:       - Hello
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 5


In [ ]:
let pstring str =
    str
    |> List.ofSeq
    |> List.map pchar
    |> sequence
    |> mapP charListToStr
    <?> str

In [ ]:
//// test

let input = fromStr "Hello"
let parser = pstring "Hello"
runOnInput parser input |> _equal (
    Success (
        "Hello",
        {
            lines = [| "Hello" |]
            position = { line = 0; column = 5 }
        }
    )
)

Success
      Item:       - Hello
      - InputState
          lines: [ Hello ]
          position: Position
            line: 0
            column: 5


In [ ]:
let whitespaceChar =
    satisfy Char.IsWhiteSpace "whitespace"

In [ ]:
let spaces = many whitespaceChar

In [ ]:
let spaces1 = many1 whitespaceChar

In [ ]:
//// test

let input = fromStr "  Hello"
let parser = spaces1 .>>. pstring "Hello"
runOnInput parser input |> _equal (
    Success (
        ([' '; ' '], "Hello"),
        {
            lines = [| "  Hello" |]
            position = { line = 0; column = 7 }
        }
    )
)

Success
      Item:       -         - [  ,   ]
        - Hello
      - InputState
          lines: [   Hello ]
          position: Position
            line: 0
            column: 7


In [ ]:
let digitChar =
    satisfy Char.IsDigit "digit"

In [ ]:
//// test

let input = fromStr "Hello"
let parser = digitChar
runOnInput parser input |> _equal (
    Failure (
        "digit",
        "Unexpected 'H'",
        {
            currentLine = "Hello"
            line = 0
            column = 0
        }
    )
)

Failure
      Item1: digit
      Item2: Unexpected 'H'
      Item3: ParserPosition
        currentLine: Hello
        line: 0
        column: 0


In [ ]:
let pint =
    let resultToInt (sign, digits) =
        let i = int digits
        match sign with
        | Some ch -> -i
        | None -> i

    let digits = manyChars1 digitChar

    opt (pchar '-') .>>. digits
    |> mapP resultToInt
    <?> "integer"

In [ ]:
//// test

run pint "-123"
|> _equal (
    Success (
        -123,
        {
            lines = [| "-123" |]
            position = { line = 0; column = 4 }
        }
    )
)

Success
      Item:       - -123
      - InputState
          lines: [ -123 ]
          position: Position
            line: 0
            column: 4


In [ ]:
let pfloat =
    let resultToFloat (((sign, digits1), point), digits2) =
        let fl = float $"{digits1}.{digits2}"
        match sign with
        | Some ch -> -fl
        | None -> fl

    let digits = manyChars1 digitChar

    opt (pchar '-') .>>. digits .>>. pchar '.' .>>. digits
    |> mapP resultToFloat
    <?> "float"

In [ ]:
//// test

run pfloat "-123.45"
|> _equal (
    Success (
        -123.45,
        {
            lines = [| "-123.45" |]
            position = { line = 0; column = 7 }
        }
    )
)

Success
      Item:       - -123.45
      - InputState
          lines: [ -123.45 ]
          position: Position
            line: 0
            column: 7


In [ ]:
let createParserForwardedToRef<'a> () =
    let dummyParser : Parser<'a> =
        let innerFn _ = failwith "unfixed forwarded parser"
        { parseFn = innerFn; label = "unknown" }

    let mutable parserRef = dummyParser

    let innerFn input =
        runOnInput parserRef input

    let wrapperParser = { parseFn = innerFn; label = "unknown" }
    
    wrapperParser, (fun v -> parserRef <- v)

In [ ]:
let (>>%) p x =
    p
    |>> fun _ -> x